In [3]:
import csv
import matplotlib.pyplot as plt
import nltk
import pandas
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB

pandas.set_option('display.max_rows', 1000)

# loading the dataset
dataset = pandas.read_csv("labeled01.csv",sep=',',usecols=['tweet','label'])

print(type(dataset))
print(f"Number of tweets: {len(dataset)}")
display(dataset)

<class 'pandas.core.frame.DataFrame'>
Number of tweets: 666


,tweet,label
0,Vaccini: il successo della Brexit premetterà a...,1
1,Aifa autorizza il vaccino AstraZeneca ma ne ra...,1
2,vaccini Mentre l'EU cerca (tra l'altro senza s...,0
3,Si andrebbe presumibilmente ben oltre l'anno. ...,2
4,"Oggi 30 gennaio , ho ricevuto la seconda dose ...",1
5,"In realtà, soprattutto nel caso di confronto c...",2
6,Quale sarebbe il vantaggio del nuovo vaccino t...,0
7,firewall76 nicolatosti1 ScaltritiLab rominafuo...,0
8,vaccino vaccini vaccinoCovid mammamia bambini ...,0
9,antonellaviol17 Dipende. Cosa succederà se avr...,2


In [4]:
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

italian_stemmer = SnowballStemmer('italian')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([italian_stemmer.stem(w) for w in analyzer(doc)])

In [8]:
# defining the pipeline

text_clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf', MultinomialNB()),
])
folds =  10
#calculating accuracies in cross-valudation
scores = cross_val_score(text_clf, dataset.tweet, dataset.label, cv=folds)
print("Accuracy MultinomialNB : %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print(scores)

Accuracy MultinomialNB : 0.50 (+/- 0.18)
[0.46268657 0.43283582 0.43283582 0.37313433 0.52238806 0.50746269
 0.66666667 0.5        0.65151515 0.5       ]
